In [9]:
from ASR import input

In [10]:
raw_data_csv = "./InputFiles/normal_strain.csv"
condition_csv= "./InputFiles/input.csv"
dummy_ch_csv = "./InputFiles/dummy_ch.csv"

In [12]:
a,b,c = input.condition(raw_data_csv=raw_data_csv, condition_csv=condition_csv, dummy_ch_csv=dummy_ch_csv)

TypeError: cannot unpack non-iterable NoneType object